In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns', None)

In [2]:
# Read that data into Python

In [3]:
numericals = pd.read_csv('numerical7_02.csv', index_col=0)
categoricals = pd.read_csv('categorical7_02.csv', index_col=0)
y = pd.read_csv('target7_02.csv', index_col=0)

In [4]:
display(numericals.shape)
display(categoricals.shape)

(95412, 319)

(95412, 12)

In [5]:
# Check the datatypes of all the columns in the data.

In [6]:
categoricals.dtypes

STATE        object
CLUSTER       int64
HOMEOWNR     object
GENDER       object
DATASRCE      int64
SOLIH       float64
VETERANS     object
RFA_2R       object
RFA_2A       object
GEOCODE2     object
DOMAIN_A     object
DOMAIN_B      int64
dtype: object

In [7]:
numericals.dtypes

ODATEDW       int64
TCODE         int64
DOB           int64
AGE         float64
INCOME      float64
             ...   
AVGGIFT     float64
CONTROLN      int64
HPHONE_D      int64
RFA_2F        int64
CLUSTER2    float64
Length: 319, dtype: object

In [8]:
# For some reason there is a unnamed column

In [9]:
display(numericals.shape)
display(categoricals.shape)

(95412, 319)

(95412, 12)

In [10]:
# Check for null values in the dataframe.

In [11]:
categoricals.isnull().sum()/len(categoricals)

STATE       0.000000
CLUSTER     0.000000
HOMEOWNR    0.000000
GENDER      0.000000
DATASRCE    0.000000
SOLIH       0.935019
VETERANS    0.890727
RFA_2R      0.000000
RFA_2A      0.000000
GEOCODE2    0.000000
DOMAIN_A    0.000000
DOMAIN_B    0.000000
dtype: float64

In [12]:
# This information will not change how the model will turn out so I will be removing it

categoricals['SOLIH'].value_counts(dropna=False)

NaN     89212
12.0     5693
0.0       296
1.0        94
2.0        75
3.0        19
4.0        16
6.0         7
Name: SOLIH, dtype: int64

In [13]:
categoricals = categoricals.drop(['SOLIH'], axis=1)

In [14]:
# Will change NaN to N

categoricals['VETERANS'].value_counts(dropna=False)

NaN    84986
Y      10426
Name: VETERANS, dtype: int64

In [15]:
categoricals['VETERANS'] = categoricals['VETERANS'].fillna('N')

In [16]:
# NEXTDATE doesn't give enough valuable information to keep in my dataset according to the documentation


df = pd.DataFrame(numericals.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

,column_name,nulls
312,NEXTDATE,9973


In [17]:
# After checking my previous lab I know this is the only nan in the numerical dataset

numericals = numericals.drop(['NEXTDATE'], axis=1)
numericals

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,5.0,5.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,4.0,7.741935,95515,0,4,39.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,18.0,15.666667,148535,0,2,1.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65

In [18]:
# Split the data into numerical and catagorical. Decide if any columns need their dtype changed.

In [19]:
categoricals

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,IL,36,U,F,3,N,L,E,C,T,2
1,CA,14,H,M,3,N,L,G,A,S,1
2,NC,43,U,M,3,N,L,E,C,R,2
3,CA,44,U,F,3,N,L,E,C,R,2
4,FL,16,H,F,3,N,L,F,A,S,2
...,...,...,...,...,...,...,...,...,...,...,...
95407,other,27,U,M,3,N,L,G,C,C,2
95408,TX,24,H,M,3,N,L,F,A,C,1
95409,MI,30,U,M,3,N,L,E,B,C,3
95410,CA,24,H,F,2,N,L,F,A,C,1


In [20]:
numericals

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,8901,0,3712,60.000000,5.0,5.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,4.0,7.741935,95515,0,4,39.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,18.0,15.666667,148535,0,2,1.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65

In [21]:
# numericals = numericals[''].value_counts(dropna=False)

In [22]:
categoricals.nunique()

STATE       12
CLUSTER     53
HOMEOWNR     2
GENDER       3
DATASRCE     3
VETERANS     2
RFA_2R       1
RFA_2A       4
GEOCODE2     4
DOMAIN_A     5
DOMAIN_B     4
dtype: int64

In [23]:
categoricals['CLUSTER'].value_counts(dropna=False) # This should be part of numericals

40    6295
35    3624
36    3617
27    3553
24    3539
49    3315
12    3000
18    2897
13    2706
30    2607
39    2601
45    2515
43    2385
11    2378
51    2322
14    2246
41    2176
44    1953
16    1947
2     1942
21    1891
8     1828
10    1804
46    1800
28    1630
17    1621
20    1568
53    1441
3     1428
34    1358
42    1358
23    1324
31    1285
22    1282
25    1275
38    1200
15    1173
1     1140
7     1076
5     1014
37     972
26     947
47     868
29     859
32     795
48     787
50     776
9      770
6      603
33     593
4      561
19     497
52     270
Name: CLUSTER, dtype: int64

In [24]:
categoricals['CLUSTER'] = categoricals['CLUSTER'].astype('category')

In [25]:
categoricals.dtypes

STATE         object
CLUSTER     category
HOMEOWNR      object
GENDER        object
DATASRCE       int64
VETERANS      object
RFA_2R        object
RFA_2A        object
GEOCODE2      object
DOMAIN_A      object
DOMAIN_B       int64
dtype: object

In [26]:
categoricals['DOMAIN_A'].value_counts(dropna=False)

R    22106
S    21924
C    19689
T    19527
U    12166
Name: DOMAIN_A, dtype: int64

In [27]:
categoricals['DOMAIN_B'].value_counts(dropna=False)

2    48356
1    28498
3    16754
4     1804
Name: DOMAIN_B, dtype: int64

In [28]:
# RFA_2R doesn't have any unique values, just a constant

categoricals = categoricals.drop(['RFA_2R'], axis=1)

In [29]:
categoricals

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,VETERANS,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B
0,IL,36,U,F,3,N,E,C,T,2
1,CA,14,H,M,3,N,G,A,S,1
2,NC,43,U,M,3,N,E,C,R,2
3,CA,44,U,F,3,N,E,C,R,2
4,FL,16,H,F,3,N,F,A,S,2
...,...,...,...,...,...,...,...,...,...,...
95407,other,27,U,M,3,N,G,C,C,2
95408,TX,24,H,M,3,N,F,A,C,1
95409,MI,30,U,M,3,N,E,B,C,3
95410,CA,24,H,F,2,N,F,A,C,1


In [30]:
# Split the data into a training set and a test set.

In [31]:
X = pd.concat([categoricals, numericals], axis=1)
y = y.drop(['TARGET_D'],axis=1)

In [32]:
X.isnull().sum()/len(X)

STATE       0.0
CLUSTER     0.0
HOMEOWNR    0.0
GENDER      0.0
DATASRCE    0.0
           ... 
AVGGIFT     0.0
CONTROLN    0.0
HPHONE_D    0.0
RFA_2F      0.0
CLUSTER2    0.0
Length: 328, dtype: float64

In [33]:
#Scale the features either by using normalizer or a standard scaler. 

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (71559, 328)
X_test shape:  (23853, 328)
y_train shape:  (71559, 1)
y_test shape:  (23853, 1)


In [35]:
y

,TARGET_B
0,0
1,0
2,0
3,0
4,0
...,...
95407,0
95408,0
95409,0
95410,1


In [36]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [37]:
print('X_train_num shape: ', X_train_num.shape)
print('X_test_num shape: ', X_test_num.shape)

X_train_num shape:  (71559, 320)
X_test_num shape:  (23853, 320)


In [38]:
# Scale the features either by using normalizer or a standard scaler.

In [39]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X_train_num)

In [40]:
X_train_norm = transformer.transform(X_train_num)
X_train_scale = pd.DataFrame(X_train_norm,columns=X_train_num.columns)
X_train_scale.head()

,DATASRCE,DOMAIN_B,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,1.0,0.000000,0.211470,0.000026,0.473841,0.525773,0.833333,0.777778,0.000000,0.0,0.525253,0.303030,0.323232,0.111111,0.000000,0.045977,0.888889,0.013850,0.015905,0.012937,1.000000,0.0,0.000000,0.494949,0.515152,0.888889,0.030303,0.010101,0.063158,0.090909,0.0,0.013889,0.000000,0.029851,0.0,0.021277,0.0,0.061856,0.0,0.0,0.034884,0.409639,0.506024,0.542169,0.432099,0.555556,0.592593,0.346667,0.424242,0.393939,0.191919,0.101010,0.222222,0.212121,0.151515,0.171717,0.121212,0.044444,0.171717,0.181818,0.292929,0.222222,0.141414,0.242424,0.050505,0.212121,0.141414,0.282828,0.585859,0.373737,0.131313,0.040404,0.626263,0.101010,0.095890,0.202020,0.364615,0.425714,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.369500,0.369833,0.692308,0.692308,0.848485,0.161616,0.989899,0.020202,0.000000,0.414141,0.838384,0.696970,0.333333,0.919192,0.090909,0.080808,0.04,0.060606,0.080808,0.151515,0.030303,0.253333,0.525253,0.171717,0.023256,0.020202,0.0,0.737374,0.959596,0.989899,1.000000,1.000000,0.030303,0.010101,0.515152,0.623529,0.611111,0.262295,0.125,0.000000,0.000000,0.000000,0.161616,0.000000,0.000000,0.000000,0.929293,0.969697,1.000000,1.000000,0.332667,0.369333,0.351333,0.392000,0.093907,0.121212,0.070707,0.090909,0.222222,0.333333,0.200000,0.06,0.016393,0.020202,0.010101,0.080808,0.090909,0.242424,0.383838,0.131313,0.08,0.010101,0.020202,0.292929,0.040404,0.393939,0.010101,0.303030,0.707071,0.040404,0.808081,0.101010,0.02439,0.02439,0.0,0.0,0.04,0.030303,0.050505,0.000000,0.191919,0.255556,0.368421,0.040404,0.787879,0.676768,0.808081,0.575758,0.777778,0.545455,0.595960,0.393939,0.909091,1.000000,0.010101,0.121212,0.191919,0.050505,0.111111,0.212121,0.000000,0.054545,0.040404,0.000000,0.161616,0.050505,0.030303,0.020202,0.020202,0.000000,0.060606,0.343434,0.050505,0.031250,0.090909,0.111111,0.050505,0.060606,0.010101,0.029851,0.060606,0.070707,0.030303,0.035088,0.111111,0.000000,0.062500,0.101010,0.737374,0.030303,0.0,0.747059,0.020202,0.131313,0.313131,0.252525,0.257143,0.151515,0.050505,0.020619,0.282828,0.090909,0.250000,0.111111,0.0,0.000000,0.0,0.222222,0.525253,0.033333,0.303030,0.232323,0.323232,0.101010,0.024096,0.060606,0.033333,0.076087,0.000000,0.000000,0.040

In [41]:
X_test_norm = transformer.transform(X_test_num)
X_test_scale = pd.DataFrame(X_test_norm,columns=X_test_num.columns)
X_test_scale.head()

,DATASRCE,DOMAIN_B,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0.5,0.333333,0.211470,0.000051,0.360556,0.639175,0.666667,0.555556,0.000000,0.0,0.313131,0.414141,0.171717,0.171717,0.080808,0.103448,0.555556,0.016656,0.017083,0.015027,1.0,0.0,0.0,0.464646,0.545455,0.010101,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.385542,0.493976,0.542169,0.419753,0.530864,0.580247,0.360000,0.353535,0.373737,0.282828,0.171717,0.181818,0.222222,0.191919,0.131313,0.070707,0.044444,0.141414,0.151515,0.262626,0.252525,0.202020,0.212121,0.050505,0.151515,0.181818,0.272727,0.555556,0.333333,0.191919,0.090909,0.424242,0.151515,0.095890,0.363636,0.343077,0.431429,0.848485,0.828283,0.010101,0.161616,0.141414,0.141414,0.000000,0.000000,0.0,0.114167,0.130167,0.230769,0.230769,0.787879,0.222222,0.929293,0.080808,0.020202,0.434343,0.767677,0.484848,0.252525,0.888889,0.121212,0.171717,0.04,0.151515,0.131313,0.222222,0.050505,0.000000,0.000000,0.010101,0.488372,0.666667,0.111111,0.030303,0.070707,0.131313,0.404040,0.787879,0.010101,0.010101,0.676768,0.694118,0.666667,0.262295,0.125,0.020202,0.141414,0.000000,0.111111,0.000000,0.111111,0.000000,0.181818,0.323232,0.585859,0.939394,0.213333,0.226667,0.252000,0.257333,0.069315,0.232323,0.090909,0.252525,0.171717,0.171717,0.127273,0.00,0.000000,0.020202,0.191919,0.070707,0.272727,0.191919,0.191919,0.090909,0.00,0.000000,0.000000,0.212121,0.181818,0.161616,0.111111,0.353535,0.656566,0.040404,0.707071,0.121212,0.207317,0.207317,0.014085,0.000000,0.0,0.010101,0.000000,0.010101,0.232323,0.255556,0.394737,0.101010,0.787879,0.686869,0.747475,0.636364,0.676768,0.616162,0.666667,0.282828,0.898990,0.686869,0.060606,0.131313,0.131313,0.020202,0.050505,0.212121,0.069767,0.054545,0.242424,0.000000,0.020202,0.050505,0.040404,0.050505,0.000000,0.000000,0.020202,0.090909,0.040404,0.015625,0.030303,0.161616,0.060606,0.070707,0.040404,0.044776,0.101010,0.121212,0.050505,0.298246,0.171717,0.080808,0.140625,0.070707,0.585859,0.020202,0.000000,0.705882,0.080808,0.121212,0.313131,0.212121,0.114286,0.080808,0.161616,0.082474,0.232323,0.045455,0.277778,0.101010,0.0,0.0,0.0,0.141414,0.313131,0.000000,0.414141,0.191919,0.171717,0.080808,0.000000,0.000000,0.000000,0.0000

In [42]:
# Afterwards, i am going to remove these columns

X_train_scale = X_train_scale.drop(['DATASRCE','DOMAIN_B','TCODE','DOB'],axis=1)
X_test_scale = X_test_scale.drop(['DATASRCE','DOMAIN_B','TCODE','DOB'],axis=1)

In [43]:
print('X_train_scale shape: ', X_train_scale.shape)
print('X_test_scale shape: ', X_test_scale.shape)      

X_train_scale shape:  (71559, 316)
X_test_scale shape:  (23853, 316)


In [44]:
# Encode the categorical features using One-Hot Encoding or Ordinal Encoding

In [45]:
X_train_cat.dtypes

STATE       object
HOMEOWNR    object
GENDER      object
VETERANS    object
RFA_2A      object
GEOCODE2    object
DOMAIN_A    object
dtype: object

In [46]:
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71559 entries, 2129 to 73193
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   STATE     71559 non-null  object
 1   HOMEOWNR  71559 non-null  object
 2   GENDER    71559 non-null  object
 3   VETERANS  71559 non-null  object
 4   RFA_2A    71559 non-null  object
 5   GEOCODE2  71559 non-null  object
 6   DOMAIN_A  71559 non-null  object
dtypes: object(7)
memory usage: 4.4+ MB


In [47]:
X_train_cat = X_train_cat.astype(str)
X_test_cat = X_test_cat.astype(str)

In [48]:
# Encode the categorical features using One-Hot Encoding or Ordinal Encoding. (train_cat, test_cat)

In [49]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first',handle_unknown = 'ignore').fit(X_train_cat)


cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
X_train_cat_encode = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
X_train_cat_encode.head(10)

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_OTHER,VETERANS_Y,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:

cols = encoder.get_feature_names_out(input_features=X_test_cat.columns)
X_test_cat_encode = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)
X_test_cat_encode.head(10)

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_OTHER,VETERANS_Y,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
print('X_train_cat_encode: ', X_train_cat_encode.shape)
print('X_train_scale: ', X_train_scale.shape)
print('X_test_cat_encode: ', X_test_cat_encode.shape)
print('X_test_scale: ', X_test_scale.shape)

X_train_cat_encode:  (71559, 25)
X_train_scale:  (71559, 316)
X_test_cat_encode:  (23853, 25)
X_test_scale:  (23853, 316)


In [52]:
X_train_transformed = pd.concat([X_train_cat_encode, X_train_scale], axis=1)
X_test_transformed = pd.concat([X_test_cat_encode, X_test_scale], axis=1)

In [53]:
print('X_train_transformed shape: ', X_train_transformed.shape)
print('X_test_transformed shape: ', X_test_transformed.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train_transformed shape:  (71559, 341)
X_test_transformed shape:  (23853, 341)
y_train shape:  (71559, 1)
y_test shape:  (23853, 1)


In [54]:
X_train_transformed.isnull().sum()/len(X_train_transformed)

nulls_percent_df = pd.DataFrame(X_train_transformed.isna().sum()/len(X_train_transformed)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage


In [55]:
# Fit a logistic regression model on the training data.

In [56]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=42, solver='lbfgs')
classification.fit(X_train_transformed, y_train)

C:\Users\13306\anaconda3\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\13306\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [57]:
predictions = classification.predict(X_test_transformed)

In [58]:
# Check the accuracy on the test data.

In [59]:
classification.score(X_test_transformed, y_test)

0.9486437764641764

In [60]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [61]:
# Check for the imbalance.

In [62]:
print(X_train_transformed.shape)
print(y_train.shape)

(71559, 341)
(71559, 1)


In [63]:
X_train_transformed = X_train_transformed.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [64]:
all_data = pd.concat([X_train_transformed, y_train], axis=1)

In [65]:
all_data.shape

(71559, 342)

In [66]:
all_data['TARGET_B'].value_counts()

0    67941
1     3618
Name: TARGET_B, dtype: int64

In [67]:
all_data.shape

(71559, 342)

In [68]:
from sklearn.utils import resample

category_0 = all_data[all_data['TARGET_B'] == 0]
category_1 = all_data[all_data['TARGET_B'] == 1]

# Undersample

In [69]:
category_0_undersampled = resample(category_0, 
                                   replace=False, 
                                   n_samples = len(category_1))

In [70]:
category_0_undersampled.shape

(3618, 342)

In [71]:
data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0)

In [72]:
data_downsampled['TARGET_B'].value_counts()

0    3618
1    3618
Name: TARGET_B, dtype: int64

In [73]:
X_down = data_downsampled.drop(['TARGET_B'],axis=1)
y_down = data_downsampled[['TARGET_B']]

In [74]:
X_down

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_OTHER,VETERANS_Y,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,ODATEDW,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
42780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.498208,0.391753,0.666667,0.555556,0.000000,0.000000,0.292929,0.393939,0.191919,0.070707,0.010101,0.011494,0.666667,0.158175,0.172642,0.153885,0.070707,0.707071,0.242424,0.494949,0.515152,0.979798,0.010101,0.010101,0.010526,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.349398,0.445783,0.481928,0.395062,0.506173,0.543210,0.386667,0.414141,0.393939,0.202020,0.151515,0.282828,0.222222,0.151515,0.090909,0.060606,0.055556,0.161616,0.181818,0.333333,0.202020,0.131313,0.151515,0.060606,0.141414,0.181818,0.313131,0.505051,0.313131,0.121212,0.030303,0.595960,0.090909,0.068493,0.272727,0.310769,0.400000,0.707071,0.646465,0.030303,0.222222,0.191919,0.151515,0.020202,0.020202,0.000000,0.147000,0.158000,0.307692,0.307692,0.757576,0.252525,0.969697,0.040404,0.070707,0.424242,0.757576,0.636364,0.343434,0.858586,0.151515,0.080808,0.04,0.060606,0.141414,0.151515,0.060606,0.306667,0.636364,0.111111,0.000000,0.010101,0.000000,0.010101,0.060606,0.282828,0.757576,0.989899,0.000000,0.040404,0.545455,0.705882,0.644444,0.245902,0.125,0.090909,0.121212,0.070707,0.050505,0.050505,0.141414,0.000000,0.242424,0.626263,0.797980,0.878788,0.257333,0.283333,0.282667,0.313333,0.087530,0.131313,0.141414,0.171717,0.262626,0.202020,0.109091,0.04,0.016393,0.010101,0.080808,0.111111,0.171717,0.303030,0.222222,0.070707,0.04,0.010101,0.020202,0.191919,0.040404,0.454545,0.050505,0.464646,0.545455,0.070707,0.808081,0.141414,0.000000,0.000000,0.000000,0.000000,0.0,0.020202,0.040404,0.010101,0.525253,0.211111,0.289474,0.020202,0.616162,0.777778,0.858586,0.696970,0.808081,0.676768,0.787879,0.676768,0.727273,0.969697,0.030303,0.101010,0.121212,0.040404,0.121212,0.171717,0.000000,0.018182,0.111111,0.020202,0.131313,0.101010,0.040404,0.060606,0.020202,0.000000,0.070707,0.262626,0.040404,0.031250,0.060606,0.161616,0.060606,0.050505,0.020202,0.029851

In [75]:
y_down

,TARGET_B
42780,0
37036,0
47629,0
56474,0
38121,0
...,...
71396,1
71429,1
71434,1
71485,1


In [88]:
LR_down = LogisticRegression(random_state=0, solver='lbfgs')
LR_down.fit(X_down, y_down)


pred = LR_down.predict(X_test_transformed)

#print("score: ",LR_down.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

C:\Users\13306\anaconda3\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


precision:  0.06684168012924072
recall:  0.5404081632653062
f1:  0.11896846077814718


C:\Users\13306\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Oversample

In [78]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [79]:
category_1_oversampled.shape

(67941, 342)

In [80]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)

In [81]:
data_upsampled['TARGET_B'].value_counts()

0    67941
1    67941
Name: TARGET_B, dtype: int64

In [82]:
X_up = data_upsampled.drop(['TARGET_B'],axis=1)
y_up = data_upsampled[['TARGET_B']]

In [90]:
LR_over = LogisticRegression(random_state=0, solver='lbfgs')
LR_over.fit(X_up, y_up)

pred = LR_over.predict(X_test_transformed)

#print("score: ",LR.score(X_test, y_test)
      
print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))


C:\Users\13306\anaconda3\lib\site-packages\sklearn\utils\validation.py:1107: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


precision:  0.06797272432081394
recall:  0.5126530612244898
f1:  0.12003058103975535


C:\Users\13306\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
